<a href="https://colab.research.google.com/github/bermanlabemory/gait_signatures/blob/main/Gait_Signatures_Script_5_Compute_Gait_Signature_Alignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Compute Gait Signature Alignment for short- and long0time predictions. 
____________
____________
**NOTE**: This file was ONLY used to select model hyperparameters and generate supplementary figures.

First, Scripts 1-4 must be run to generate the models that are analyzed here.
____________
____________
**Steps:** 
Short-time alignment:
1. Load short-time externally- and self-driven internal parameters. Note that the externally- and self-driven domains are the same - no phase-averaging needed.
2. Project into the same low-D basis using Principal Components Analysis (PCA)
2. Compute R-squared of the signatures
4. Save the R2 values
____________
Long-time alignment
1. Load long-time externally- and self-driven internal parameters. Here, the domains on the externally- and self-driven signatures differ, such that we need to phase average the data to compare trajectories.
2. Project into the same low-D basis using Principal Components Analysis (PCA). This gives gait signature trajectories in time.
2. Estimate gait phase and phase average externally- and self-driven signatures separately
2. Compute R-squared of the phase-averaged gait signatures
4. Save the R2 values
___________
Note that, for short and long-time alignment, Models that fail to predict rhythmic oscillations are given NaN values. An arbitrary negative R2 value could also be computed.
____________
____________

**Created by**: Michael C. Rosenberg

**Date**: 09/22/22

**Step 0**: Mount (connect to) your google drive folder where you want to save the simulation results and model parameters.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
#drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# check python version 
from platform import python_version
print(python_version())

# check tensorflow version
import tensorflow as tf
print(tf.__version__)

# Check memory
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

3.7.15


**Step 1**: Import necessary packages and functions to develop model

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
# from keras.layers.recurrent import LSTM # Deprecated :(
from tensorflow.python.keras.layers.recurrent import LSTM
from sklearn.model_selection import train_test_split
import sklearn.model_selection as model_selection
import matplotlib.pyplot as plt
import math
import keras as k
import pandas as pd
import numpy as np
from copy import copy
import scipy.io
from sklearn.decomposition import PCA

from scipy.signal import find_peaks
from scipy import interpolate
from scipy.special import iv
from numpy import sin,cos,pi,array,linspace,cumsum,asarray,dot,ones
from pylab import plot, legend, axis, show, randint, randn, std,lstsq

from sklearn.manifold import MDS
import csv
import os
from tqdm import tqdm 

In [ ]:
%cd /content/drive/My Drive/NeuromechanicsLab/GaitSignatures/

# Ensure fourierseries.py is in the pathway
!ls -l fourierseries.py

/content/drive/My Drive/NeuromechanicsLab/GaitSignatures
-rw------- 1 root root 7259 May 29  2020 fourierseries.py


In [ ]:
import fourierseries
import util
import phaser
import dataloader
# Preprocess data for a single subject - to be send to modeling frameworks
def find_phase(k):
    """
    Detrend and compute the phase estimate using Phaser
    INPUT:
      k -- dataframe
    OUTPUT:
      k -- dataframe
    """
    #l = ['hip_flexion_l','hip_flexion_r'] # Phase variables = hip flexion angles
    y = np.array(k)
    print(y.shape)
    y = util.detrend(y.T).T
    print(y.shape)
    phsr = phaser.Phaser(y=y)
    k[:] = phsr.phaserEval(y)[0,:]
    return k

In [ ]:
def vonMies(t,t_0, b):
    out = np.exp(b*np.cos(t-t_0))/(2*pi*iv(0, b))
    return out

**Step 2**: Load module in Google Drive

In [ ]:
# The path to save the models and read data from
path = '/content/drive/My Drive/NeuromechanicsLab/GaitSignatures/'

# Insert the directory
import sys
sys.path.insert(0,path)

**Step 3**: Load in data and specify variables/parameters

In [ ]:
# Non-changing variables 

# number of trials in dataset 
trialnum = 72 # 72 total trials

# number of samples in each trial
trialsamp = 1500

# number of features collected per trial
feats = 6

#Batch size - same as the number of traintrials
batch_size = trialnum

# Number of Layers
numlayers = 1

# Choose the number of iterations to train the model- if this script has been run previously enter a value greater than was 
# inputted before and rerun the script. 
finalepoch = 10000

# load the input data/kinematics
datafilepath = path + 'PareticvsNonP_RNNData.csv' #input data
all_csvnp = np.loadtxt(datafilepath,delimiter=',').T

# reshape all the input data into a tensor
all_inputdata_s = all_csvnp.reshape(trialnum,trialsamp,feats) 
csvnp = all_inputdata_s
print('original input data shape is: '+ str(all_csvnp.shape ))
print('input data reshaped is: '+ str(all_inputdata_s.shape))

original input data shape is: (108000, 6)
input data reshaped is: (72, 1500, 6)


**Step 4**: Develop list of model architectures and corresponding variables to train. This step also generates list of folder names and pathways where the models will be saved and accessed later.

In [ ]:
# generate a list of models and corresponding parameters to test 
test_model_nodes = [128, 256,512,1024] 
seqs = [249,499,749] #lookback parameter

test_model_nodes = [ 512] 
seqs = [499] #lookback parameter

# run multiple model architechtures many times to test stability of cost function outputs
runs = 1 # Number of times to train recurrent neural network (RNN) models, starting from random initial conditions. We used this for hyperparameter selection
test_model_seq = np.repeat(seqs, runs) # Specify each model's hyperparameters

count = np.arange(runs)

All_nodes = np.empty([0,1], dtype='int')
All_seq = np.empty([0,1],dtype='int')
All_valseg = np.empty([0,1],dtype='int')
All_trainseg = np.empty([0,1],dtype='int')
All_modelname = []
All_mod_name = []
count = np.empty([0,1],dtype='int'); #initialize model run -- this serves as the model run ID number
ct = 0
for a in test_model_nodes:
  for b in test_model_seq:
    count = np.append(count,  ct + 1 )
    #if statement for valseg, trainseg based on sequence length
    if int(b) == 249:
      trainseg = 4
      valseg = 2
    elif int(b) == 499: 
      trainseg = 2
      valseg = 1
    elif int(b) == 749:
      trainseg = 1
      valseg = 1

    # Store resulting model structures and training plans
    All_nodes = np.append(All_nodes, a) 
    All_seq = np.append(All_seq, int(b))
    All_valseg = np.append(All_valseg, valseg)
    All_trainseg = np.append(All_trainseg, trainseg)
    All_modelname = np.append(All_modelname, 'UNIT_' + str(a) + '_LB_' + str(b) + '_run_' + str(count[-1]) + '/' )
    All_mod_name = np.append(All_mod_name, 'UNIT_' + str(a) + '_LB_' + str(b) + '_run_' + str(count[-1]) )

    if ct+1 < runs:
      ct += 1
    else:
      ct = 0

30
['UNIT_256_LB_499_run_1' 'UNIT_256_LB_499_run_2' 'UNIT_256_LB_499_run_3'
 'UNIT_256_LB_499_run_4' 'UNIT_256_LB_499_run_5' 'UNIT_256_LB_499_run_6'
 'UNIT_256_LB_499_run_7' 'UNIT_256_LB_499_run_8' 'UNIT_256_LB_499_run_9'
 'UNIT_256_LB_499_run_10' 'UNIT_256_LB_749_run_1' 'UNIT_256_LB_749_run_2'
 'UNIT_256_LB_749_run_3' 'UNIT_256_LB_749_run_4' 'UNIT_256_LB_749_run_5'
 'UNIT_256_LB_749_run_6' 'UNIT_256_LB_749_run_7' 'UNIT_256_LB_749_run_8'
 'UNIT_256_LB_749_run_9' 'UNIT_256_LB_749_run_10' 'UNIT_512_LB_499_run_1'
 'UNIT_512_LB_499_run_2' 'UNIT_512_LB_499_run_3' 'UNIT_512_LB_499_run_4'
 'UNIT_512_LB_499_run_5' 'UNIT_512_LB_499_run_6' 'UNIT_512_LB_499_run_7'
 'UNIT_512_LB_499_run_8' 'UNIT_512_LB_499_run_9' 'UNIT_512_LB_499_run_10']
/content/drive/My Drive/NeuromechanicsLab/GaitSignatures/UNIT_256_LB_499_run_1/UNIT_256_LB_499_run_1_bestwhole.h5


## Compute SHORT-time gait signature alignment
1. Load short-time self- and externally-driven internal states (H & C values) from Script 4. These samples are paired.
2. Run PCA to project all data into the same basis.
2. Compute the mean squared error between self- and ext-driven signatures (note: need to use the same samples for all initializations -> need to extract non-zero indices that are common across all initializations for a fair comparison)
2. Compare r-squared of prediction accuracy (note: MSE increases and r2 decreases with increasing number of samples (N). Both change by a factor of N2/N1). Report N2/N1






In [ ]:
# Pre-allocate arrays
err = np.empty((len(All_mod_name), 1))
sigAlign = np.zeros((trialnum, 1))
R2 = np.zeros((trialnum, 1))  # THIS IS OUR ALIGNMENT OUTCOME


for j in range(len(All_mod_name)): # For each model
  plt.figure(figsize = (25,25))

  # make folder to store model
  newfoldpath = path + All_mod_name[j]

  # extract path to store each model and generated data
  savepath = path + All_modelname[j]
  mod_name = All_mod_name[j]
  print('Working on: ' + mod_name + ' model ' + str(j) + ' / ' + str(len(All_mod_name)))

  # Number of Units
  numunits = All_nodes[j]
  test_ind = np.arange(0, len(all_inputdata_s), 1)  # run all input data through
  ext_drive= trialsamp
  tot_len = len(test_ind) # len of all trials

  # Load Short-time gait signature H & C's and concatenate
  ExtSelfHCs = scipy.io.loadmat(savepath + mod_name + '_selfExtDriveHC_shortTime.mat')
  ExtHC = ExtSelfHCs['HC_ext_concat']
  SelfHC = ExtSelfHCs['HC_self_concat']

  # Project into a common low-D basis using PCA
  pca = PCA(n_components=numunits*2) #initialize
  pca.fit(ExtHC)
  ExtSig = pca.transform(ExtHC) # X is projected on the first principal components previously extracted from a training set - data is centered here
  SelfSig = pca.transform(SelfHC) 

  # Compute gait sig alignment and R2 for each trials
  predLen = int(np.shape(SelfHC)[0]/trialnum)
  for tr in range(trialnum):
    inds = range( (tr*predLen),  ((tr+1)*predLen) )
    # Get this metric as a sanity check and a way to discard bogus trials later
    sigAlign[tr,0] = np.linalg.norm( np.reshape(ExtSig[inds,:] - SelfSig[inds,:], (-1,1))) # Norm of difference b/t Externally and self-driven signatures

    # Compute R-squared (R2) - THIS IS OUR OUTCOME
    SSR = np.sum( (np.reshape(ExtSig[inds,:],(-1,1)) - np.reshape(SelfSig[inds,:], (-1,1) ) )**2 )
    SST = np.sum( (np.reshape(ExtSig[inds,:],(-1,1)) - np.mean(np.reshape(ExtSig[inds,:], (-1,1) )) )**2 )
    R2[tr,0] = 1 - SSR/SST
    if R2[tr,0] > 0.99:
      R2[tr,0] = np.nan

    # Plot
    ax = plt.subplot(12,6,tr+1)
    ax.plot(ExtSig[inds,0],'-')
    ax.plot(SelfSig[inds,0],'-')
    if tr > 65: 
      plt.xlabel('Time')
    else:
      ax.set_xticklabels([])
      ax.set_xticks([])
    
    plt.title('R2 = ' + str(np.round(R2[tr,0],2)) + ' | ' + 'Alignment = ' + str(np.round(sigAlign[tr,0],0)))

  scipy.io.savemat(savepath + mod_name + '_shortTimeAlignment.mat',{'EuclideanAlignment': sigAlign,'R2': R2})
  plt.savefig(savepath + 'selfDrivenPredictions.png', dpi = 150)
  plt.show()

Working on: UNIT_256_LB_499_run_1 model 0 / 30


KeyboardInterrupt: ignored

<Figure size 1800x1800 with 0 Axes>

## Long-time predictions

1. Load ext & self-diven gait signatures
2. Compute PCs for externally driven signatures. This is necessary because self-driven signatures that converge to zero can cause problems.

3. Project externally- and self-driven sigs into that the same PC basis

3. Phase average the ext-driven signatures
5. Compute long-time gait sig alignment (alignment of the average signatures)

In [ ]:
for j in range(28,len(All_mod_name)):
  R2 = np.zeros((trialnum, 1)) # Pre-allocate R-squared array)

  # Create figure for sanity chech
  plt.figure(figsize= (20,40))

  # make folder to store model
  newfoldpath = path + All_mod_name[j]

  # extract path to store each model and generated data
  savepath = path + All_modelname[j]
  mod_name = All_mod_name[j]
  print('Working on: ' + mod_name + ' model ' + str(j) + ' / ' + str(len(All_mod_name)))

  # Number of Units
  numunits = All_nodes[j]
  test_ind = np.arange(0, len(all_inputdata_s), 1)  # run all input data through
  ext_drive= trialsamp
  tot_len = len(test_ind) # len of all trials

  # Load H & C and concatenate
  ExternalDriveHCs_testset = scipy.io.loadmat(savepath + mod_name + '_extdriveHCs_testset.mat')
  ExternalDriveHCs_testset = ExternalDriveHCs_testset['ext_drive_sigs']
  SelfDriveHCs_testset = scipy.io.loadmat(savepath + mod_name + '_selfdriveHCs.mat')
  SelfDriveHCs_testset = SelfDriveHCs_testset['self_drive_sigs']

  # Concatenate -[Ext,Self] for each trial
  CombinedExternal_Self_HCs = np.zeros( (2*np.shape(ExternalDriveHCs_testset)[0], np.shape(ExternalDriveHCs_testset)[1]) )
  for jj in range(72):
    inds1 = range(jj*1500,(jj+1)*1500)
    inds2 = range(jj*3000,(jj+1)*3000)
    CombinedExternal_Self_HCs[inds2,:] = np.concatenate( (ExternalDriveHCs_testset[inds1,:], SelfDriveHCs_testset[inds1,:]), axis = 0)

  # Project into a common low-D basis using PCA
  pca = PCA(n_components=numunits*2) #initialize
  pca.fit(ExternalDriveHCs_testset)
  X_reduction = pca.transform(CombinedExternal_Self_HCs) # X is projected on the first principal components previously extracted from a training set - data is centered here
  np.save(savepath + mod_name + '_pca_vaf_extSelfDrive.npy', pca.explained_variance_ratio_[0:6])

  ###########################################################################
  ###########################################################################
  ###########################################################################

  # Get  and allocate arrays
  HC_CellArray = np.empty(shape=[tot_len, ext_drive*2, X_reduction.shape[1]])
  start = 0
  last = ext_drive*2 # length of trials
  print('X_reduction shape = ' + str(np.shape(X_reduction)))
  for p in range(tot_len):
      HC_CellArray[p]= X_reduction[start:last]
      start = start+ext_drive*2
      last = last+ext_drive*2

  # Generate phase averaged signatures
  print('generating phase averaged signatures')
  numSegments = 100 # Number of samples in one cycle
  lim = tot_len
  L = HC_CellArray.shape[2] # Number of PCs to phase average

  # Allocate external and self-driving arrays
  PhaseAveragedPCsExt = np.empty(shape=[lim, L, numSegments])
  PhaseAveragedPCs_shiftExt = np.empty(shape=[lim, L, numSegments])
  Phase_VariablesExt = np.empty(shape=[lim, ext_drive])
  PC_ShiftsExt = np.empty(shape=[lim]) # store phase shift values

  PhaseAveragedPCsSelf = np.empty(shape=[lim, L, numSegments])
  PhaseAveragedPCs_shiftSelf = np.empty(shape=[lim, L, numSegments])
  Phase_VariablesSelf = np.empty(shape=[lim, ext_drive])
  PC_ShiftsSelf = np.empty(shape=[lim]) # store phase shift values

  cyclephase = [] # Phase of the gait cycle - needed to align the cycles 

  # Average orbits initialization
  phaseVals = np.linspace(0, 2*pi, numSegments, endpoint=True) # phases that we want our phase averaged orbits to correspond to
  kappa = 20

  # Run phase estimates and phase average 
  badInds = []
  for a in range(lim): #for length of all the trials 
    print(['processing trial: ' + str(a)]) 
    dats=[] # reset variables after each trial - kinematics
    dats2 = [] # reset the HC params
    allsigs = np.empty(shape=[numSegments,])
    all_phase_var = np.empty(shape=[numSegments,]) 
    PhaseAvgPCs = np.empty(shape=[numSegments])
    all_cyclephase = []
    raw = HC_CellArray[a][:,0:3].T # Only use 1st 3 PCs (of HCs),
    raw2 = HC_CellArray[a][:,0:L].T  # extract all the HC data
    for b in range(6): #Shai's code does duplication- works better than without duplicating data
        dats.append(raw-raw.mean(axis=1)[:,np.newaxis]) #center the HCs data for phase estimation -- this centers each one separately
        dats2.append(raw2-raw2.mean(axis=1)[:,np.newaxis])
    rHC = raw2 #uncentered 

    # Run Phaser ONLY if the self-driven prediction doesn't converge to a trivial result (this breaks Phaser)
    if a < 1000: # np.std(dats[0][0,:1500]) / np.std(dats[0][0,2250:]) < 1.5: # Only run phaser is variance of the end of the self-driven prediction is greater than 1/2 that of externally-driven prediction
      phr = phaser.Phaser(dats) # use centered data from 1st 3 PCs
      phi = [ phr.phaserEval( d ) for d in dats ] # extract phase
      phi2  = (phi[0].T % (2*pi)); # Take modulo s.t. phase ranges from [0,2*pi]
      
      # Split into externally and self-driven sets
      rHCExt = rHC[:,0:ext_drive]
      rHCSelf = rHC[:,ext_drive:]
      phi2Ext = phi2[0:ext_drive,0]
      phi2Self = phi2[ext_drive:,0]

      # find average orbits of ext and self-driven sigs separately
      avgOrbitsExt = np.zeros((numSegments,L)) #initialize avg orbits
      avgOrbitsSelf = np.zeros((numSegments,L)) #initialize avg orbits
      phasesExt = np.reshape(phi2Ext,[ext_drive,]) # Extract phase (external drive)
      phasesSelf = np.reshape(phi2Self,[ext_drive,]) # Self-drive

      for c in range(numSegments): #number of points in final average orbit/phase averaging
        # Externally-driven
          vonMiesCurrent = vonMies(phasesExt,phaseVals[c],kappa) # for each value in the num of phase points calculate current
          sumVal = np.sum(vonMiesCurrent)
          for d in range(L):  # Average each feature
              data = np.reshape(rHCExt[d,:],[ext_drive,1])
              avgOrbitsExt[c,d] = np.sum(data.T*vonMiesCurrent)/sumVal # phase point (row), feature of PC (column) -- this is generating a value for the 1st phase point of each feature
        # Self-driven
          vonMiesCurrent = vonMies(phasesSelf,phaseVals[c],kappa) # for each value in the num of phase points calculate current
          sumVal = np.sum(vonMiesCurrent)
          for d in range(L):  # Average each feature
              data = np.reshape(rHCSelf[d,:],[ext_drive,1])
              avgOrbitsSelf[c,d] = np.sum(data.T*vonMiesCurrent)/sumVal # phase point (row), feature of PC (column) -- this is generating a value for the 1st phase point of each feature        

      # Store phase-averaged results and phase shift the to PC1 max = zero phase
      # Use the same phase-shift for both ext- and self-driven signatures
      # Externally driven #######################
      PhaseAveragedPCsExt[a] = avgOrbitsExt.T # transform to match overall saving structure as before
      phi2Ext = phi2Ext.reshape(trialsamp,)
      Phase_VariablesExt[a] = phi2Ext # store phase variables for all cycles/features - may need to phase shift these 
      # Phase Shift - This aligns the Gait signatures in phase
      PC1_maxloc = np.argmax(PhaseAveragedPCsExt[a][0]) # only PC1 value (1st max if repeated) - ONLY BASED ON EXT-DRIVEN
      Data2Shift = PhaseAveragedPCsExt[a]
      NewP = np.roll(Data2Shift, -PC1_maxloc,axis=1) #shift back to orgin
      PC_ShiftsExt[a] = PC1_maxloc
      PhaseAveragedPCs_shiftExt[a] = NewP

      # Self driven #######################
      PhaseAveragedPCsSelf[a] = avgOrbitsSelf.T # transform to match overall saving structure as before
      phi2Self = phi2Self.reshape(trialsamp,)
      Phase_VariablesSelf[a] = phi2Self # store phase variables for all cycles/features - may need to phase shift these 
      # Phase Shift
      Data2Shift = PhaseAveragedPCsSelf[a]
      NewP = np.roll(Data2Shift, -PC1_maxloc,axis=1) #shift back to orgin
      PC_ShiftsSelf[a] = PC1_maxloc
      PhaseAveragedPCs_shiftSelf[a] = NewP

      # Compute R2
      SSR = np.sum( (np.reshape(PhaseAveragedPCs_shiftExt[a],(-1,1)) - np.reshape(PhaseAveragedPCs_shiftSelf[a], (-1,1) ) )**2 )
      SST = np.sum( (np.reshape(PhaseAveragedPCs_shiftExt[a],(-1,1)) - np.mean(np.reshape(PhaseAveragedPCs_shiftExt[a], (-1,1) )) )**2 )
      R2[a,0] = 1 - SSR/SST
      if R2[a,0] > 0.99:
        R2[a,0] = np.nan

    else: 
      print('Trial ' + str(a) + ' is junk.')
      badInds.append(a)

    # Plot phase-averaged PCs
    plt.subplot(12,6,a+1)
    print(np.shape(PhaseAveragedPCs_shiftSelf[a]))
    plt.plot(PhaseAveragedPCs_shiftExt[a][0,:], label = 'Externally-driven')
    plt.plot(PhaseAveragedPCs_shiftSelf[a][0,:], label = 'Self-driven')
    if a > 65:
      plt.xlabel('Percent gait phase')
    if (a % 6) == 0:
      plt.ylabel('PC 1 of gait signature')

    plt.title('Trial ' + str(a) + ' | R2 = ' + str(np.round(R2[a,0], 2))  )

  # Save gait sig alignment
  scipy.io.savemat(savepath + mod_name + '_LongTimeAligment.mat',{'R2': R2})


  plt.savefig(savepath + mod_name + '_PC1_predictions.png', dpi = 150)
  plt.show()